## Описание проекта

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

**Признаки**
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

**Целевой признак**
- Exited — факт ухода клиента

**План работы над проектом**

1. Подготовка данных
2. Исследование задачи
3. Борьба с дисбалансом
4. Тестирование модели
5. Выводы

In [1]:
!pip install -U scikit-learn
# Почему-то иногда ломалась тетрадка, если вручную не переустановить пакеты. Подсказал преподователь с проектов.

In [2]:
# Подключим все необходимые библиотеки:

import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [3]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv('D:/Churn.csv')

## Подготовка данных

Изучим имеющиеся данные:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [6]:
# Поверим датасет на наличие пропусков:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [7]:
# Поверим датасет на явные дубликаты:
data.duplicated().sum()

0

Чтобы продолжить дальнейший анализ необходимо устранить следующие недочёты в имеющихся данных:
1. Удалить столбцы `RowNumber`, `CustomerId` и `Surname`. Первый дублирует индексы, второй является уникальным Id пользователей, а третий - фамилиями пользователей. Все эти данные никак не помогут в построении модели и предсказаний;
2. Избавиться от пропусков в столбце `Tenure`;
3. Заменим категориальные признаки в столбцах `Geography` и `Gender` на числовые. 

### Удаление лишних столбцов

In [8]:
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis='columns', inplace=True)

### Заполнение пропусков

Стобец `Tenure` обозначает количество недвижимости у клиента. Предположим, что пропуск в данном случае подразумевает её отсутствие и заполним пропуски 0:

In [9]:
data.fillna(data['Tenure'].median(), inplace=True)

### Создание выборок

In [10]:
# В признаки модели (features) отнесём все данные выборки, за исключением столбца "Exited", 
# т.к. это целевой признак (target).

features = data.drop(['Exited'], axis=1)
target = data['Exited']

Целевой признак выборки представлен категорией из 0 и 1 - это задача **бинарной классификации**.

Отделим 60% генеральной выборки для обучения модели, 20% для валидации и 20% для окончательной проверки дееспесобности выбранной модели:

In [11]:
# С помощью метода train_test_split разделим генеральную выбрку на тренировочную (*_train), валидационную (*_valid) 
# и тестовую (*_test)

features_train, features_rest, target_train, target_rest = \
    train_test_split(features, target, test_size=0.4, random_state=0)

features_valid, features_test, target_valid, target_test = \
    train_test_split(features_rest, target_rest, test_size=0.5, random_state=0)

In [12]:
# Убедимся, что пропорция выборок 3:1:1 соблюдена:
display(f'Обучающая выборка составила: {features_train.shape[0]/features.shape[0]*100} %')
display(f'Валидационная выборка составила: {features_valid.shape[0]/features.shape[0]*100} %')
display(f'Тестовая выборка составила: {features_test.shape[0]/features.shape[0]*100} %')

'Обучающая выборка составила: 60.0 %'

'Валидационная выборка составила: 20.0 %'

'Тестовая выборка составила: 20.0 %'

### Замена категориальных признаков

Для замены категориальных признаков числовыми воспользуемся методом one-hot encoding:

In [13]:
cols = ['Geography', 'Gender'] #названия столбцов, кот.надо разбить на числа
 
enc = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
 
#Cоздание объекта DF:
features_train_enc = pd.DataFrame(data=enc.fit_transform(features_train[cols]),
                                      columns=enc.get_feature_names_out(cols)) 
 
# Объединим датафрейм с преобразованными столбцами с исходным: 
features_train = features_train.reset_index(drop=True)
target_train = target_train.reset_index(drop=True)
new_features_train = pd.concat([features_train_enc, features_train], axis=1)
 
#Уберем категориальные столбцы из нового датафрейма:
new_features_train.drop(cols, axis=1, inplace=True) 

# Выведем первые 10 строчек 
new_features_train.head(10)

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Geography_Germany,Geography_Spain,Gender_Male,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0.0,1.0,1.0,579,43,2.0,145843.82,1,1,1,198402.37
1,0.0,1.0,0.0,652,40,7.0,100471.34,1,1,1,124550.88
2,0.0,1.0,1.0,652,37,5.0,0.00,2,1,0,68789.93
3,0.0,0.0,1.0,645,25,0.0,174400.36,1,1,0,42669.37
4,0.0,0.0,1.0,613,36,9.0,131307.11,1,0,0,83343.73
5,0.0,1.0,1.0,756,38,6.0,119208.85,1,1,0,169763.89
6,0.0,1.0,1.0,712,45,6.0,112994.65,1,0,0,198398.68
7,0.0,0.0,0.0,592,31,5.0,84102.11,2,0,1,116385.24
8,1.0,0.0,1.0,636,49,6.0,113599.74,2,1,0,158887.09
9,1.0,0.0,1.0,553,53,5.0,127997.83,1,1,0,165378.66


In [14]:
features_valid_encoder = pd.DataFrame(data=enc.transform(features_valid[cols]), 
                                      columns=enc.get_feature_names_out(cols)) 
features_valid = features_valid.reset_index(drop=True)
new_features_valid = pd.concat([features_valid_encoder, features_valid], axis=1) 
new_features_valid.drop(cols, axis=1, inplace=True)

features_test_encoder = pd.DataFrame(data=enc.transform(features_test[cols]), 
                                      columns=enc.get_feature_names_out(cols)) 
features_test = features_test.reset_index(drop=True)
new_features_test = pd.concat([features_test_encoder, features_test], axis=1) 
new_features_test.drop(cols, axis=1, inplace=True)

Таким образом модель легко проанализирует пол изучаемого объекта и страну проживания.

Следующим шагом будет определние целевого признака и создание тренировочной, валидационной и тестовой выборок: 

На этапе предобработки данных:
1. Отсортированы лишние столбцы;
2. Заполнены пропуски;
3. Категориальные показатели заменены на числовые;
4. Сформированы обучающая, валидационная и тестовая выборки.

Можно переходить к следующему этапу.

## Исследование задачи

In [15]:
# Рассмотрим дисбаланс классов по целевым признакам:

target.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

В среднем на 1 положительный класс выходит 4 отрицательных. Попробуем обучить модели без исправление баланса.

Для нашего исследования мы воспользуемся тремя моделями:
- логистческая регрессия;
- случайный лес;
- ближайшие соседи.

Для каждой модели подберём наиболее точные параметры, и выведем их с указанием F1 (среднее гармоническое полноты и точности).

### Логистическая регрессия

In [16]:
model = LogisticRegression(random_state=0, solver = 'liblinear')
model.fit(new_features_train, target_train)
regression_predict = model.predict(new_features_valid)
regression_probabilities = model.predict_proba(new_features_valid)[:, 1]
regression_auc_roc = roc_auc_score(target_valid, regression_probabilities)


display('Точность модели "Логистическая регрессия": {:.1f}%'.format(model.score(new_features_valid, target_valid)*100), 
        "F1: {:.2f}%".format(f1_score(target_valid, regression_predict)*100),
        "AUC-ROC метрика: {:.2f}%".format(regression_auc_roc*100))

'Точность модели "Логистическая регрессия": 78.5%'

'F1: 12.22%'

'AUC-ROC метрика: 67.24%'

### Случайный лес

Исследуем наилучшие параметры для модели, чтобы вывести наибольшую точность:

In [17]:
best_result = 0
best_est = 0
best_depth = 0
best_f1 = 0
best_auc_roc = 0
for est in range(50, 201, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model.fit(new_features_train, target_train)
        result = model.score(new_features_valid, target_valid)
        forest_predict = model.predict(new_features_valid)
        f1 = f1_score(target_valid, forest_predict)
        forest_probabilities = model.predict_proba(new_features_valid)[:, 1]
        roc = roc_auc_score(target_valid, forest_probabilities)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth
            best_f1 = f1
            best_auc_roc = roc

display('Наибольшая точность модели "Случайный лес": {:.1f}%'.format(best_result*100), 
        "При количестве деревьев: {}".format(best_est), 
        "Глубине: {}".format(best_depth),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Случайный лес": 87.6%'

'При количестве деревьев: 160'

'Глубине: 14'

'И F1: 63.52%'

'AUC-ROC метрика: 87.42%'

### Ближайшие соседи

In [18]:
best_result = 0
best_neigbors = 0
best_f1 = 0
best_auc_roc = 0
for neighbors in range(1, 11):
    model = KNeighborsClassifier(n_neighbors=neighbors) 
    model.fit(new_features_train, target_train)
    result = model.score(new_features_valid, target_valid)
    neighboor_predict = model.predict(new_features_valid)
    f1 = f1_score(target_valid, neighboor_predict)
    neighboor_probabilities = model.predict_proba(new_features_valid)[:, 1]
    roc = roc_auc_score(target_valid, neighboor_probabilities)
    if result > best_result:
        best_result = result
        best_neigbors = neighbors
        best_f1 = f1
        best_auc_roc = roc

display('Наибольшая точность модели "Ближайшие соседи": {:.1f}%'.format(best_result*100), 
        "При кол-ве соседей: {}".format(best_neigbors),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Ближайшие соседи": 79.0%'

'При кол-ве соседей: 10'

'И F1: 3.23%'

'AUC-ROC метрика: 53.69%'

Имеем слудеющие выводы:
1. Модель **"Логистическая регрессия"**:
    - Точность = 78.5%
    - Величина F1 = 12.22%
    - AUC-ROC метрика: 67.24%
    
    
2. Модель **"Случайный лес"**:
    - Точность = 87.6%
    - Кол-во деревьев = 160
    - Глубина = 14
    - Величина F1 = 63.52%
    - AUC-ROC метрика: 87.42%
    
    
3. Модель **"Ближайшие соседи"**:
    - Точность = 79%
    - Кол-во соседей = 10
    - Величина F1 = 3.23%
    - AUC-ROC метрика: 53.69%
    
Наиболее точной моделью, как по показателю score, F1 и метрике AUC-ROC оказалась модель "Случайный лес".
Попробуем обучить модели уже на более сбалансированной выборке и сравним результаты.

## Борьба с дисбалансом

### Глобавльный параметр class_weight

Наиболее простой способ сбалансировать выборку - указать каждой модели глобальный параметр `class_weight = balanced`.

In [19]:
model = LogisticRegression(random_state=0, solver = 'liblinear', class_weight='balanced')
model.fit(new_features_train, target_train)
regression_probabilities = model.predict_proba(new_features_valid)[:, 1]
regression_auc_roc = roc_auc_score(target_valid, regression_probabilities)

display('Точность модели "Логистическая регрессия" при указанном class_weight: {:.1f}%'.format(model.score(new_features_valid, target_valid)*100), 
        "F1: {:.2f}%".format(f1_score(target_valid, regression_predict)*100),
       "AUC-ROC метрика: {:.2f}%".format(regression_auc_roc*100))

'Точность модели "Логистическая регрессия" при указанном class_weight: 69.5%'

'F1: 12.22%'

'AUC-ROC метрика: 76.74%'

In [20]:
best_result = 0
best_est = 0
best_depth = 0
best_f1 = 0
best_auc_roc = 0
for est in range(50, 201, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(new_features_train, target_train)
        result = model.score(new_features_valid, target_valid)
        forest_predict = model.predict(new_features_valid)
        f1 = f1_score(target_valid, forest_predict)
        forest_probabilities = model.predict_proba(new_features_valid)[:, 1]
        roc = roc_auc_score(target_valid, forest_probabilities)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth
            best_f1 = f1
            best_auc_roc = roc

display('Наибольшая точность модели "Случайный лес" при указанном class_weight: {:.1f}%'.format(best_result*100), 
        "При количестве деревьев: {}".format(best_est), 
        "Глубине: {}".format(best_depth),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Случайный лес" при указанном class_weight: 86.6%'

'При количестве деревьев: 130'

'Глубине: 15'

'И F1: 61.71%'

'AUC-ROC метрика: 87.06%'

Модель "Ближайшие соседи" не имеет глобального параметра `class_weight` и увиличивать её показатель F1 придётся другими способами.

По результатам проведённой работы видно, что показатель F1 в обоих случаях увеличился, ценой снижения точности. Особенно это заметно на примере "Логистической регрессии". 

### Уменьшение частоты класса 0

С помощью функции downsample изменим частоту класса 0 в генеральной выборке и обучим на ней наши модели:

In [21]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=0)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=0)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=0)
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(new_features_train, target_train, 0.25)

# Выведем на экран объём положительных и отрицательных целевых признаков после работы функции:
target_downsampled.value_counts()

1    1209
0    1198
Name: Exited, dtype: int64

In [22]:
model = LogisticRegression(random_state=0, solver = 'liblinear')
model.fit(features_downsampled, target_downsampled)
regression_predict_downsampled = model.predict(new_features_valid)
f1 = f1_score(target_valid, regression_predict_downsampled)
regression_probabilities_downsampled = model.predict_proba(new_features_valid)[:, 1]
regression_auc_roc = roc_auc_score(target_valid, regression_probabilities_downsampled)

display('Точность модели "Логистическая регрессия" при уменьшении частоты класса 0: {:.1f}%'.format(model.score(new_features_valid, target_valid)*100), 
        "F1: {:.2f}%".format(f1*100),
       "AUC-ROC метрика: {:.2f}%".format(regression_auc_roc*100))

'Точность модели "Логистическая регрессия" при уменьшении частоты класса 0: 64.8%'

'F1: 44.83%'

'AUC-ROC метрика: 70.45%'

In [23]:
best_result = 0
best_est = 0
best_depth = 0
best_f1 = 0
best_auc_roc = 0
for est in range(50, 201, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(features_downsampled, target_downsampled)
        result = model.score(new_features_valid, target_valid)
        forest_predict_downsampled = model.predict(new_features_valid)
        f1 = f1_score(target_valid, forest_predict_downsampled)
        forest_probabilities_downsampled = model.predict_proba(new_features_valid)[:, 1]
        roc = roc_auc_score(target_valid, forest_probabilities_downsampled)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth
            best_f1 = f1
            best_auc_roc = roc

display('Наибольшая точность модели "Случайный лес" при уменьшении частоты класса 0: {:.1f}%'.format(best_result*100), 
        "При количестве деревьев: {}".format(best_est), 
        "Глубине: {}".format(best_depth),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Случайный лес" при уменьшении частоты класса 0: 80.5%'

'При количестве деревьев: 90'

'Глубине: 11'

'И F1: 62.92%'

'AUC-ROC метрика: 87.02%'

In [24]:
best_result = 0
best_neigbors = 0
best_f1 = 0
best_auc_roc = 0
for neighbors in range(1, 11):
    model = KNeighborsClassifier(n_neighbors=neighbors) 
    model.fit(features_downsampled, target_downsampled)
    result = model.score(new_features_valid, target_valid)
    neighboor_predict_downsampled = model.predict(new_features_valid)
    f1 = f1_score(target_valid, neighboor_predict_downsampled)
    neighboor_probabilities_downsampled = model.predict_proba(new_features_valid)[:, 1]
    roc = roc_auc_score(target_valid, neighboor_probabilities_downsampled)
    if result > best_result:
        best_result = result
        best_neigbors = neighbors
        best_f1 = f1
        best_auc_roc = roc

display('Наибольшая точность модели "Ближайшие соседи" при уменьшении частоты класса 0: {:.1f}%'.format(best_result*100), 
        "При кол-ве соседей: {}".format(best_neigbors),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Ближайшие соседи" при уменьшении частоты класса 0: 64.7%'

'При кол-ве соседей: 2'

'И F1: 25.68%'

'AUC-ROC метрика: 51.93%'

Итого, при уменьшении частоты класса 0:

1. Модель **"Логистическая регрессия"**:
    - Точность = 64.8%
    - Величина F1 = 44.83%
    - AUC-ROC метрика: 70.45%
    
    
2. Модель **"Случайный лес"**:
    - Точность = 80.5%
    - Величина F1 = 62.92%
    - AUC-ROC метрика: 87.02%
    
    
3. Модель **"Ближайшие соседи"**:
    - Точность = 64.7%
    - Величина F1 = 25.68%
    - AUC-ROC метрика: 51.93%
    
Попробуем пойти в обратном направлении и увеличить частоту класса 1 в генеральной выборке:

### Увеличение частоты класса 1

In [25]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=0)
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(new_features_train, target_train, 4)

# Выведем на экран объём положительных и отрицательных целевых признаков после работы функции:
display(target_upsampled.value_counts())

1    4836
0    4791
Name: Exited, dtype: int64

In [26]:
model = LogisticRegression(random_state=0, solver = 'liblinear')
model.fit(features_upsampled, target_upsampled)
regression_predict_upsampled = model.predict(new_features_valid)
regression_probabilities_upsampled = model.predict_proba(new_features_valid)[:, 1]
f1 = f1_score(target_valid, regression_predict_upsampled)

regression_auc_roc = roc_auc_score(target_valid, regression_predict_upsampled)

display('Точность модели "Логистическая регрессия" при увеличении частоты класса 1: {:.1f}%'.format(model.score(new_features_valid, target_valid)*100), 
        "F1: {:.2f}%".format(f1*100),
       "AUC-ROC метрика: {:.2f}%".format(regression_auc_roc*100))

'Точность модели "Логистическая регрессия" при увеличении частоты класса 1: 65.4%'

'F1: 45.17%'

'AUC-ROC метрика: 66.93%'

In [27]:
best_result = 0
best_est = 0
best_depth = 0
best_f1 = 0
best_auc_roc = 0
for est in range(50, 201, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(features_upsampled, target_upsampled)
        result = model.score(new_features_valid, target_valid)
        forest_predict_upsampled = model.predict(new_features_valid)
        f1 = f1_score(target_valid, forest_predict_upsampled)
        forest_probabilities_upsampled = model.predict_proba(new_features_valid)[:, 1]
        roc = roc_auc_score(target_valid, forest_probabilities_upsampled)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth
            best_f1 = f1
            best_auc_roc = roc

display('Наибольшая точность модели "Случайный лес" при увеличении частоты класса 1: {:.1f}%'.format(best_result*100), 
        "При количестве деревьев: {}".format(best_est), 
        "Глубине: {}".format(best_depth),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Случайный лес" при увеличении частоты класса 1: 85.3%'

'При количестве деревьев: 170'

'Глубине: 15'

'И F1: 63.07%'

'AUC-ROC метрика: 87.06%'

In [28]:
best_result = 0
best_neigbors = 0
best_f1 = 0
best_auc_roc = 0
for neighbors in range(1, 11):
    model = KNeighborsClassifier(n_neighbors=neighbors) 
    model.fit(features_upsampled, target_upsampled)
    result = model.score(new_features_valid, target_valid)
    neighboor_predict_upsampled = model.predict(new_features_valid)
    f1 = f1_score(target_valid, neighboor_predict_upsampled)
    neighboor_probabilities_upsampled = model.predict_proba(new_features_valid)[:, 1]
    roc = roc_auc_score(target_valid, neighboor_predict_upsampled)
    if result > best_result:
        best_result = result
        best_neigbors = neighbors
        best_f1 = f1
        best_auc_roc = roc

display('Наибольшая точность модели "Ближайшие соседи" при увеличении частоты класса 1: {:.1f}%'.format(best_result*100), 
        "При кол-ве соседей: {}".format(best_neigbors),
        "И F1: {:.2f}%".format(best_f1*100),
        "AUC-ROC метрика: {:.2f}%".format(best_auc_roc*100))

'Наибольшая точность модели "Ближайшие соседи" при увеличении частоты класса 1: 68.0%'

'При кол-ве соседей: 1'

'И F1: 22.14%'

'AUC-ROC метрика: 51.00%'

Наиболее удачной моделью оказался "Случайный лес", обученный на выборке с увеличенной частотой класса 1. Проверим его на тестовой выборке.

## Тестирование модели

In [33]:
model = RandomForestClassifier(n_estimators=170, max_depth=15, random_state=0)
model.fit(features_upsampled, target_upsampled)
forest_predict_test = model.predict(new_features_test)
forest_probabilities_test = model.predict_proba(new_features_test)[:, 1]
f1 = f1_score(target_test, forest_predict_test)

display('Точность модели "Случайный лес" на тестовой выборке: {:.1f}%'.format(model.score(new_features_test, target_test)*100))
display("Величина F1: {:.2f}%".format(f1*100))

'Точность модели "Случайный лес" на тестовой выборке: 83.9%'

'Величина F1: 59.95%'

Оценим метрики AUC-ROC, чтобы понять насколько точно модель спрогнозировла показатели в сравнении с целевой тестовой выборкой.

In [34]:
auc_roc_result = roc_auc_score(target_test, forest_probabilities_test)

display("AUC-ROC: {:.2f}%".format(auc_roc_result*100))

'AUC-ROC: 45.10%'

Наша модель на 5% хуже предсказвыает показатели, чем случайная модель.

## Выводы

С целью обнаружения наилучшей модели прогнозирования целевого показателя (уйдёт ли клиент из банка или нет) на этапе предобработки данных:

- Отсортированы лишние столбцы;
- Заполнены пропуски;
- Категориальные показатели заменены на числовые;
- Сформированы обучающая, валидационная и тестовая выборки в отношении 3:1:1.

В ходе исследования задачи были выбраны наилучшие глобальные параметры для 3 моделей обучения:
- Логистическая регрессия;
- Случайный лес;
- Ближайшие соседи.

Модели были поочердно обучены как на несбалансированной выборке, так и на сбалансированной разными способами: через гиперпараметр, с помощью уменьшения частоты класса 0 и увеличения класса 1. Наилучшие показатели точности и величины F1 (средней гармонической полноты и точности) оказались у модели "Случайный лес", обученной на выборке с уменьшиной частотой результативного признака класса 0:

- Точность: 83.95%;
- Величина F1: 59.95%;
- Метрика AUC-ROC: 45.10%.